In [1]:
versioninfo()

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) Silver 4110 CPU @ 2.10GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


In [2]:
using LinearAlgebra
using Distributions

cd("../src")

include("hchol.jl")
include("mvn.jl")
include("truncnorm.jl")
include("hcmvn.jl")
include("cmvn.jl")

CMVN

In [3]:
# test for hchol
A = rand(Normal(0,1), 8, 8)
A = A'A
A = Symmetric(A)
m = 2
B, UV = hchol(A, m)
displayTree(UV)

L = uncompress(B, UV)
@assert A ≈ L*L'

Block: UnitRange{Int64}[5:8, 1:4], Rank: 4
Block: UnitRange{Int64}[3:4, 1:2], Rank: 2
Block: UnitRange{Int64}[7:8, 5:6], Rank: 2


## Test for mvns

In [4]:
dim = 16
m = 4
d = 2
a1 = repeat([-10.0], dim)
b1 = repeat([0.0], dim)
ns = 10

10

### 1. $\Sigma = I$

In [5]:
A = convert(Matrix{Float64}, LinearAlgebra.I(dim))
Σ = A*A'
L = cholesky(Σ).L
N = 1000
tol = 1e-8

1.0e-8

#### probabilities

In [6]:
mvn(L, a1, b1, ns = ns, N = N, tol = tol, μ = ones(Float64, length(a1)))

1.6116522399570697e-13

In [7]:
p, _ = CMVN(Symmetric(Σ), a1 - ones(Float64, length(a1)), b1 - ones(Float64, length(a1)), d; ns = ns, N = N, tol = tol)
p

1.611652239957063e-13

In [8]:
HMVN(a1, b1, Symmetric(Σ), m, ns = ns, N = N, tol = tol, μ = ones(Float64, length(a1)))

1.6116522399570704e-13

In [9]:
HCMVN(a1, b1, Symmetric(Σ), d, m, μ = ones(Float64, length(a1)))

1.6116522399570649e-13

In [10]:
mvn(L, a1, b1; ns = ns, N = N, tol = tol, μ = b1)

1.5258789062500003e-5

In [11]:
p, _ = CMVN(Symmetric(Σ), a1 - b1, b1 - b1, d; ns = ns, N = N, tol = tol)
p

1.52587890625e-5

In [12]:
HMVN(a1, b1, Symmetric(Σ), m; ns = ns, N = N, tol = tol, μ = b1)

1.5258789062500007e-5

In [13]:
HCMVN(a1, b1, Symmetric(Σ), d, m, μ = b1)

1.5258789062500007e-5

#### expectation

In [14]:
expt_tnorm(a1, b1, L)

16-element Array{Float64,1}:
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654

In [15]:
_, y = CMVN(Symmetric(Σ), a1, b1, d)
y

16-element Array{Float64,1}:
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654
 -0.7978845608028654

### 2. $\Sigma = AA'$

$A_{i,j} \sim N(0,1)$

In [16]:
A = rand(Normal(0,1), dim, dim)
Σ = A*A'
L = cholesky(Σ).L
N = 1000
tol = 1e-8

1.0e-8

#### probabilities

In [17]:
mvn(L, a1, b1, ns = ns, N = N, tol = tol, μ = ones(Float64, length(a1)))

1.2498899687744652e-62

In [18]:
p, _ = CMVN(Symmetric(Σ), a1 - ones(Float64, length(a1)), b1 - ones(Float64, length(a1)), d; ns = ns, N = N, tol = tol)
p

4.1033098479628055e-7

In [19]:
HMVN(a1, b1, Symmetric(Σ), m, ns = ns, N = N, tol = tol, μ = ones(Float64, length(a1)))

0.0

In [20]:
HCMVN(a1, b1, Symmetric(Σ), d, m, μ = ones(Float64, length(a1)))

3.081223826086738e-8

In [21]:
mvn(L, a1, b1; ns = ns, N = N, tol = tol, μ = b1 .- 2)

0.011140200192415347

In [22]:
p, _ = CMVN(Symmetric(Σ), a1 - (b1 .- 2), b1 - (b1 .- 2), d; ns = ns, N = N, tol = tol)
p

0.00967350746754874

In [23]:
HMVN(a1, b1, Symmetric(Σ), m; ns = ns, N = N, tol = tol, μ = b1 .- 2)

0.007506584881674045

In [24]:
HCMVN(a1, b1, Symmetric(Σ), d, m, μ = b1 .- 2)

0.006174649413946383

#### expectation

In [25]:
expt_tnorm(a1, b1, L)

16-element Array{Float64,1}:
 -2.4337367611045315 
 -4.315854703175674  
 -2.647917139771913  
 -4.361950700986237  
 -2.97593475347354   
 -4.759618441542154  
  0.7445989682684159 
 -4.559216513615207  
 -3.5166086538169345 
 -3.179076049506679  
 -1.6902307868986166 
 -5.202453262520512  
  0.6138086657871246 
 -3.898196792269704  
 -1.2917500610174228 
  0.34024705489143736

In [28]:
_, y = CMVN(Symmetric(Σ), a1, b1, dim)
y

16-element Array{Float64,1}:
 -2.4337367611045315 
 -4.315854703175674  
 -2.647917139771913  
 -4.361950700986237  
 -2.97593475347354   
 -4.759618441542154  
  0.7445989682684159 
 -4.559216513615207  
 -3.5166086538169345 
 -3.179076049506679  
 -1.6902307868986166 
 -5.202453262520512  
  0.6138086657871246 
 -3.898196792269704  
 -1.2917500610174228 
  0.34024705489143736

In [27]:
_, y = CMVN(Symmetric(Σ), a1, b1, d)
y

16-element Array{Float64,1}:
 -4.0884756624899925 
 -2.9854470001990454 
 -5.322416720478782  
 -1.4912938461104255 
 -1.7884410735853598 
 -1.6387915238196236 
 -3.2321198290091995 
 -0.6605450743611053 
 -3.2055708036661525 
 -9.421655497420408  
 -4.443196145927975  
  0.6816879161778726 
 -1.4606983219933214 
 -1.0587965019564398 
  0.5141295488397521 
 -0.33874555798882544